In [1]:
from speedy_utils import *

In [5]:
import time


def f(s):
    # sleep for s seconds
    time.sleep(s)
    print(s)
    return s

In [6]:
inputs = [1, 2, 3, 4, 5]
outputs = multi_thread(f, inputs)

1
2


2025-01-07 17:07:28.071 | WARNING  | speedy_utils.multi_worker.thread:multi_thread:129 - Execution manually interrupted


3
4
5


In [4]:
outputs

[1, 2, 3, None, None]